# Class 9 - Starter Code

In [ ]:
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import grid_search
from sklearn import metrics
from sklearn import linear_model
from sklearn import dummy

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid", font_scale=1)
%matplotlib inline

## Part 1. Load Dataset

#### 1.1 Let's load the dataset and check the first five rows

In [ ]:
# load data
df = pd.read_csv('../../assets/dataset/admissions.csv')

# check head
df.head()

## Part 2. Pre-Process Data

#### 2.1 Check and remove missing values
**Reading**: Read Pandas docs on handling missing values:
[http://pandas.pydata.org/pandas-docs/stable/missing_data.html](http://pandas.pydata.org/pandas-docs/stable/missing_data.html)

In [ ]:
# check for missing values in each column before dropping
print "Missing values:"
print df.isnull().sum()

In [ ]:
# drop missing values if there are any
if df.isnull().sum().sum():
    print "There are missing values"
    df = df.dropna()
    print "Missing values dropped"

In [ ]:
# check for missing values in each column after dropping
print "Missing values:"
print df.isnull().sum()

#### 2.2 Check and convert all data types to numerical

In [ ]:
# check data types
print df.dtypes

In [ ]:
# Get numerical columns
num_cols = set(df.dtypes[((df.dtypes=="int64")|(df.dtypes=="float64"))].index)
non_cols = set(df.columns)-num_cols

print "Numerical columns:"
print num_cols
print "Non-numerical columns:"
print non_cols

# here all columns are numeric; no need to convert

#### 2.3 Check and create dummy variables for categorical features
**Reading**: API Docs for `get_dummies()`:
[http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)

In [ ]:
# create dummy variables for `prestige` feature
# this check allows this code to be run multiple times
if 'prestige' in df.columns:
    # get dummy variables for prestige
    df = df.join(pd.get_dummies(df['prestige'], prefix='prestige'))
    # remove prestige column
    df.drop(['prestige'], axis=1, inplace=True)

In [ ]:
# check for newly added columns
df.head()

## Part 3. Cross Validation

#### 3.1 Create separate training and test sets
**Reading**:

Read Scikit docs on cross validation:
[http://scikit-learn.org/stable/modules/cross_validation.html](http://scikit-learn.org/stable/modules/cross_validation.html)

Read Scikit docs on `sklearn.cross_validation.train_test_split()`:
[http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html)

In [ ]:
# set X and y
X = df.drop(['admit'], axis=1)
y = df['admit']

In [ ]:
# create separate training and test set with 60/40 train/test split
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
# check size of training set
print X_train.shape, y_train.shape

# check size of test set
print X_test.shape, y_test.shape

## Part 4. Build Model

#### 4.1 Build Logistic Regression Classifier
**Reading**: Read Scikit docs for `sklearn.linear_model.LogisticRegression`: [http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
# instantiate lm classifier using default params
lm = linear_model.### FILL IN ###

# train lm classifier on training set
lm.fit(X_train, y_train)

#### 4.2 Evaluate Model
**Reading**: Read Scikit docs on evaluating models: [http://scikit-learn.org/stable/modules/model_evaluation.html](http://scikit-learn.org/stable/modules/model_evaluation.html)

In [ ]:
# check model accuracy on test set
accuracy = lm.score(X_test, y_test)
print "Accuracy: " + str(accuracy)

**Reading**: Read Scikit docs on confusion matrix: [http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

In [ ]:
# Get confusion matrix on test set
y_pred = lm.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

ax = plt.axes()
sns.heatmap(cm_normalized, annot=True)
ax.set_ylabel('True')
ax.set_xlabel('Pred')
plt.show()

print cm

**Reading**: Read API Docs for [sklearn.metrics.roc_curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)

In [ ]:
# Plot ROC curve and get AUC score

# train and predict using dummy model
dm = dummy.DummyClassifier()
dm.fit(X_train, y_train)
predict_proba_dm_test = dm.predict_proba(X_test).T[1]

# train and predict using logstic model
lm = linear_model.LogisticRegression()
lm.fit(X_train, y_train)
predict_proba_lm_test = lm.predict_proba(X_test).T[1]

# plot ROC curve
ax = plt.subplot(111)
vals = metrics.roc_curve(y_test, predict_proba_dm_test)
ax.plot(vals[0], vals[1])
vals = metrics.### FILL IN ###
ax.plot(vals[0], vals[1])

ax.set(title='Area Under the Curve for Prediction', ylabel='True Positive Rate', xlabel='False Positive Rate', xlim=(0, 1), ylim=(0, 1))

# get AUC score
print "ROC AUC Score: " + str(metrics.roc_auc_score(y_test, lm.predict(X_test)))

## Part 5. Tune Model

#### 5.1 Perform Grid Search for `C`
**Reading**: Read Scikit docs for `sklearn.grid_search.GridSearchCV`:
[http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html)

In [ ]:
# set list of values to grid search over
c = range(1, 150)
params = {'C': c}

# perform grid search using list of values
gs = grid_search.GridSearchCV(
    estimator=linear_model.LogisticRegression(),
    param_grid=params)
gs.fit(X_train, y_train)

# get best value to use
print gs.### FILL IN ###

In [ ]:
# plot search values vs. grid scores
plt.plot(c, [s[1] for s in gs.grid_scores_])

#### 5.2 Update model using best `n_neighbors`

In [ ]:
# instantiate lm classifier using updated params
lm = linear_model.LogisticRegression(### FILL IN ###

# train updated lm classifier on training set
lm.fit(X_train, y_train)

#### 5.3 Evaluate updated model

In [ ]:
# check updated model accuracy on test set
accuracy = lm.score(X_test, y_test)
print "Accuracy: " + str(accuracy)

In [ ]:
# Get confusion matrix on test set
y_pred = lm.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

ax = plt.axes()
sns.heatmap(cm_normalized, annot=True)
ax.set_ylabel('True')
ax.set_xlabel('Pred')
plt.show()

print cm

In [ ]:
# Plot ROC curve and get AUC score of updated model

# train and predict using dummy model
dm = dummy.DummyClassifier()
dm.fit(X_train, y_train)
predict_proba_dm_test = dm.predict_proba(X_test).T[1]

# train and predict using logstic model
lm1 = linear_model.LogisticRegression()
lm1.fit(X_train, y_train)
predict_proba_lm1_test = lm1.predict_proba(X_test).T[1]

# train and predict using improved logstic model
lm2 = ### FILL IN ###
lm2.fit(X_train, y_train)
predict_proba_lm2_test = ### FILL IN ###

# plot ROC curve
ax = plt.subplot(111)
vals = metrics.roc_curve(y_test, predict_proba_dm_test)
ax.plot(vals[0], vals[1], label='Dummy Model')
vals = metrics.roc_curve(y_test, predict_proba_lm1_test)
ax.plot(vals[0], vals[1], label='Logistic Regression')
vals = ### FILL IN ###
ax.plot(vals[0], vals[1], label='Improved Logistic Regression')

ax.set(title='Area Under the Curve for Prediction', ylabel='True Positive Rate', xlabel='False Positive Rate', xlim=(0, 1), ylim=(0, 1))
plt.legend(loc='lower right')

# get AUC score
print "ROC AUC Score for lm1: " + str(metrics.roc_auc_score(y_test, lm1.predict(X_test)))
print "ROC AUC Score for lm2: " + ### FILL IN ###